- bitflyerのAPI設定
- ティッカーとか板情報の取得

In [19]:
import configparser
from datetime import datetime as dt
import os
import threading
import time
import websocket

import ccxt
import numpy as np
import pandas as pd
import pytz
from tqdm import tqdm

In [2]:
# API

conf = configparser.ConfigParser()
conf.read('../../../data/bot/settings.ini')

api_key = conf['bitflyer']['api_key']
api_secret = conf['bitflyer']['api_secret']

# CCXT
bitflyer = ccxt.bitflyer()

bitflyer.apiKey = api_key
bitflyer.secret = api_secret

In [3]:
# 5分で300回以上リクエストを送るとエラー
# https://lightning.bitflyer.com/docs?lang=ja#http-private-api

In [4]:
# 口座残高の確認

# bitflyer.private_get_getbalance()

## ticker

In [5]:
params = {
    'product_code': 'BTC_JPY'
}

In [6]:
ticker = bitflyer.public_get_getticker(params=params)
ticker

{'product_code': 'BTC_JPY',
 'state': 'RUNNING',
 'timestamp': '2021-03-17T18:30:12.437',
 'tick_id': 39123004,
 'best_bid': 6163372.0,
 'best_ask': 6166160.0,
 'best_bid_size': 0.22,
 'best_ask_size': 1e-06,
 'total_bid_depth': 798.61931575,
 'total_ask_depth': 660.5264746,
 'market_bid_size': 0.0,
 'market_ask_size': 0.0,
 'ltp': 6162334.0,
 'volume': 19413.99439873,
 'volume_by_product': 6268.91036798}

In [7]:
bid = ticker['best_bid']
ask = ticker['best_ask']
ltp = ticker['ltp']

# 実際のspreadと一致することを確認
(ask - bid) / ltp * 100

0.045242598015622006

## 板

In [8]:
# bitflyer.public_get_getboard(params)

## 約定データ(bitflyer版)

In [9]:
# 取得対象
symbol = 'BTC_JPY'
market = 'Spot' # 現物
before = 0      # 初期値

In [12]:
def utcstr2jpystr(utc_str: str) -> str:
    utc_tz = pytz.timezone('UTC')
    jst_tz = pytz.timezone('Asia/Tokyo')
    # マイクロ秒切り捨て
    utc_str = utc_str[:19]
    utc_datetime = dt.strptime(utc_str, "%Y-%m-%dT%H:%M:%S")
    utc_datetime = utc_tz.localize(utc_datetime)
    jst_datetime = utc_datetime.astimezone(jst_tz)
    jst_str = dt.strftime(jst_datetime, "%Y-%m-%dT%H:%M:%S")
    return jst_str

In [16]:
# 約定データの保存先
path = '../../../data/crypto/bitflyer/executions/'

# すでに持っているデータ
try:
    executions = os.listdir(path+symbol)
except:
    # ディレクトリが無いなら作る
    os.mkdir(path+symbol)
    executions = os.listdir(path+symbol)
    
if '.ipynb_checkpoints' in executions:
    executions.remove('.ipynb_checkpoints')

# TODO:過去に取得してるなら差分だけ取得
if executions:
    print('already have some executions data..')
# まだ約定データを1個も持ってない場合
else:
    print('not have any executions data..')
    df_exec_list = []
    before = 0 # 初期値
    while True:
        params = {
            'product_code': symbol,
            'market': market,
            'count': 500,
            'before': before,
        }
        execution = bitflyer.public_get_getexecutions(params=params)
        # 次のループの取得開始ID
        before = execution[-1]['id']
        df_exec = pd.DataFrame(execution)\
                    .sort_values('id')\
                    .set_index('id')
        df_exec_list.append(df_exec)
        # 約定データ数が溜まったら保存
        # 100万件溜まったら保存
        if len(df_exec_list) == 1000000 / 500:
            df_executions = pd.concat(df_exec_list)
            # 昇順にソート
            df_executions = df_executions.sort_values('id', ascending=True)
            start_id = str(df_executions.index[0])
            end_id = str(df_executions.index[-1])
            filename = symbol + '_' + start_id + '_' + end_id + '.csv'
            df_executions['exec_date'] = list(map(lambda x: utcstr2jpystr(x), df_executions['exec_date']))
            df_executions.to_csv(path + symbol + '/' + filename)
            # 初期化
            df_exec_list = []
            print(filename)
        # ループ終了処理
        if True:
            pass
        
        time.sleep(1.0)

already have some executions data..


## 約定履歴からのローソク足作成 

In [42]:
symbol = 'BTC_JPY'

durations = [
    '1min',
    '5min',
    '15min',
    '30min',
    '1H',
    '4H',
    '1D',
    '1W',
    '1M',
]

# D: 毎日
# H: 毎時
# Tまたはmin: 毎分
# S: 毎秒
# Lまたはms: 毎ミリ秒
# Uまたはus: 毎マイクロ秒
# N: 毎ナノ秒
# 参考: https://note.nkmk.me/python-pandas-time-series-freq/

In [43]:
df_exec_1 = pd.read_csv('../../../data/crypto/bitflyer/executions/BTC_JPY/BTC_JPY_2174147013_2184143666.csv')
df_exec_2 = pd.read_csv('../../../data/crypto/bitflyer/executions/BTC_JPY/BTC_JPY_2184143667_2196129395.csv')
df_exec_3 = pd.read_csv('../../../data/crypto/bitflyer/executions/BTC_JPY/BTC_JPY_2196129396_2200914238.csv')
df_exec_list = [df_exec_1, df_exec_2, df_exec_3] # 古い順

In [45]:
for duration in tqdm(durations):
    ohlcv_list = []
    for df_exec in df_exec_list:
        df_exec['exec_date'] = pd.to_datetime(df_exec['exec_date'])
        df_exec = df_exec.set_index('exec_date')[['side',
                                                  'size',
                                                  'price']]
        df_ohlcv = df_exec.resample(duration)\
                                .agg({'price': 'ohlc',
                                      'size' : 'sum'}).ffill()
        df_ohlcv.columns = ['open','high','low','close','volume']
        ohlcv_list.append(df_ohlcv)
        
    ohlcv = pd.concat(ohlcv_list)
    ohlcv.to_csv('../../../data/crypto/bitflyer/ohlcv/' +
                 symbol + '/' +
                 'bf-{}-{}-{}-{}.csv'\
                .format(symbol,
                        duration,
                        ohlcv.index[0],
                        ohlcv.index[-1]))

100%|██████████| 9/9 [00:03<00:00,  2.34it/s]


In [37]:
df_exec.head()

,side,size,price
exec_date,,,
2021-02-17T17:56:58,BUY,0.01084,5394180.0
2021-02-17T17:56:58,BUY,0.00916,5394180.0
2021-02-17T17:56:58,BUY,0.13100,5394536.0
2021-02-17T17:56:58,BUY,0.00974,5396904.0
2021-02-17T17:56:59,SELL,0.02000,5394150.0


## 板データ収集

In [18]:
# 以下のコードでも取得出来るけど呼び出し制限回数がある
# 従ってリアルタイムAPIを使って板情報の差分を更新し続けなくてはいけない

# bitflyer.public_get_getboard()